In [1]:
import datasets

In [2]:
ds = datasets.load_from_disk("/mnt/disks/persist/user_comments", keep_in_memory=True)
ds = ds.filter(lambda x: len(x["posts"]) > 0, num_proc=64)
#ds.save_to_disk("/mnt/disks/persist/user_comments_filtered")

Filter (num_proc=64):   0%|          | 0/256852 [00:00<?, ? examples/s]

In [4]:
ds

Dataset({
    features: ['user', 'posts', 'depressed_label'],
    num_rows: 251210
})

In [6]:
def format_post_as_text(posts):
    return "\n\n".join(f"Post from /r/{post['subreddit']}:\n{post['body']}" for post in posts)

def add_text_format(batch):
    batch["text"] = [format_post_as_text(sample) for sample in batch["posts"]]
    return batch

new_ds = ds.map(add_text_format, remove_columns=["user", "posts"], batched=True, num_proc=64)

In [ ]:
split = new_ds.train_test_split(test_size=0.15, seed=42)
test_validation = split["test"].train_test_split(test_size=0.5, seed=42)
ds_dict = datasets.DatasetDict({
    "train": split["train"],
    "validation": test_validation["train"],
    "test": test_validation["test"]
})